In [25]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import random as rnd
import statsmodels.api as sm
import numpy.linalg as la
import sympy
from sympy import series, Symbol
from sympy.functions import sin, cos, exp
from sympy.plotting import plot
import numpy.random as random
from datetime import datetime 

# Decrypting Substitution Cipher

In [52]:
n_runs = 7
n_iters = 4000

lowercase = 'abcdefghijklmnopqrstuvwxyz'
encrypted = "m it vbeh yjmbl. qbl lgb tfwlgo iwn hbieo zftb, lgbh ziw lirb wflgmwx keft tb, lgbh ziw lirb wflgmwx tfeb. m it of iqfwb, iwn of umlgfjl gfdb lgil m ziw zfwkefwl lgbt umlgfjl kbie. lgb qmkb lgil gio cfewb tb lgefjxg lgbob hbieo mo olmqq mw th giwno iwn th bhbo. ugblgbe m givb ojcnjbn ml, m rwfu wfl. cjl of qfwx io ml mo lgbeb ml umqq obbr mlo fuw uih fjl, gbbnqboo fk lgb umqq lgil mo umlgmw tb. gb kbqq mw fzlfcbe wmwblbbw bmxglbbw fw i nih lgil uio of yjmbl iwn olmqq fw lgb ugfqb kefwl lgil lgb ieth ebdfel zfwkmwbn mlobqk lf lgb omwxqb obwlbwzb iqq yjmbl fw lgb ubolbew kefwl. gb gin kiqqbw kfeuien iwn qih fw lgb bielg io lgfjxg oqbbdmwx. ljewmwx gmt fvbe fwb oiu lgil gb zfjqn wfl givb ojkkbebn qfwx; gmo kizb gin iw badeboomfw fk ziqt, io lgfjxg iqtfol xqin lgb bwn gin zftb."
bigrams_df = pd.read_csv('bigrams.csv', index_col=0) / 100000

In [42]:
def log_f(bgrm_df, msg):
    '''
    Parameters
    ----------
    bgrm_df : DataFrame with an index of bigrams 
              and a 'count' column.
    msg : A string of valud characters.
    Returns
    -------
    A log-score for the msg.
    '''    
    # Find pairs in the decrypted (using key) msg
    msg_df = pd.DataFrame() 
    for c1,c2 in zip(msg[:-1], msg[1:]): 
        if (not c1==' ') and (not c2==' '): 
            try: 
                msg_df.loc[c1+c2, 'count'] = msg_df.loc[c1+c2, 'count']+1
            except: 
                msg_df.loc[c1+c2, 'count'] = 1
                
    # For each bigram in the msg update log_f_score.    
    # If the bigram appears 0 times in the reference corpus,  
    #     subtract some small integer (log(k)<0 when k<1)          
    log_f_score = 0             
    for b in msg_df.index: 
        try: # succeed if bgrm_df.loc[b][0] > 0
            log_f_score += np.log(bgrm_df.loc[b][0])*msg_df.loc[b][0]
        except:
            log_f_score += -7 # bgrm_df.loc[b][0] = 0
            
    return log_f_score

In [43]:
def encrypt(msg, ref, key): 
    '''
    Parameters
    ----------
    msg : string of valid characters
    ref : string of N possible original characters
    key : string of N possible substitute characters 
    Returns
    -------
    Encrypted (substituted) message
    
    To Decrypt
    ----------
    Send key first and ref second, 
    i.e., encrypt(msg, key, ref)
    '''
    key_dict = dict(zip(ref, key))
    encryp = '' 
    for let in msg: 
        try: 
            encryp += key_dict[let]
        except: 
            encryp += let
    return encryp 

In [44]:
for _ in range(n_runs): 
    # Generate a random current substitution key
    # and initialize the current score to zero.
    cur_key = ''.join(rnd.sample(lowercase, len(lowercase)))
    cur_log_f = 0 

    for itr in range(n_iters):
        # Create a new key by swapping two random letters in the current key.
        c1, c2 = rnd.sample(cur_key, 2)
        new_key = cur_key.replace(c1,'*').replace(c2,c1).replace('*',c2) 
        
        # Decrypt with the new key and score the newly decrypted message.
        new_decrypted = encrypt(encrypted, new_key, lowercase)
        new_log_f = log_f(bigrams_df, new_decrypted) 
        
        # If new score > current score, accept the new key (current=new).
        # Else, with probability (new score)/(current score) accept the new key.
        if new_log_f > cur_log_f: # redundant condition: just for clarity
            cur_key = new_key
            cur_log_f = new_log_f
        else:
            r = random.uniform(0,1)
            if r < np.exp(new_log_f-cur_log_f): 
                cur_key = new_key
                cur_log_f = new_log_f
        if itr%500==0:       
            print('(%d) log(f) = %.2f , '%(itr,cur_log_f) + new_decrypted[:61] )
            
    # Print (or save) the current key and decrypted message.         
    print('------------------------------------------------------------')
    print('Proposed key:', cur_key, '\n')
    print(encrypt(encrypted, cur_key, lowercase))  
    print('------------------------------------------------------------\n\n')
          

<ipython-input-42-b7d9f3cdfcab>:27: RuntimeWarning: divide by zero encountered in log
  log_f_score += np.log(bgrm_df.loc[b][0])*msg_df.loc[b][0]


(0) log(f) = 0.00 , f up tyhj kvfyx. lyx xoy pimxon umz jyuhn cipy, xoyj cum xuay
(500) log(f) = 4929.94 , o rm kwlb enowu. twu upw masupg rsd bwrlg hamw, upwb hrs uryw
(1000) log(f) = 5072.39 , o rm kelb wnoei. sei ice magicd rgy berld hame, iceb hrg irve
(1500) log(f) = 5122.79 , t am zerb gntei. sei ice molicd aly beard home, iceb hal iave
(2000) log(f) = 5139.75 , l am jerp cnlei. sei ike motikd aty peard home, ikep hat iave
(2500) log(f) = 5149.06 , l am verp kslei. nei ice moticd aty peard home, icep hat iaxe
(3000) log(f) = 5147.38 , l am verp knlei. sei ice moticd aty peard home, icep hat iawe
(3500) log(f) = 5149.06 , l am vebp knlei. sei ice moticd aty peabd home, icep hat iaxe
------------------------------------------------------------
Proposed key: iugobkczlsymtjfhaeqwxvdrnp 

l am verp knlei. sei ice moticd aty peard home, icep hat iaxe toicltu from me, icep hat iaxe toicltu more. l am do asote, aty do bliconi cowe icai l hat hotfroti icem bliconi fear. ice slfe icai cad 

(500) log(f) = 5080.82 , i of wpur zcips. nps stp falstm old rpoum hafp, stpr hol sokp
(1000) log(f) = 5248.52 , i ab demy xuies. les she bonshr anv yeamr gobe, shey gan sake
(1500) log(f) = 5415.76 , i am very quiet. let the months and years bome, they ban take
(2000) log(f) = 5427.71 , i am very quiet. let tbe montbs and years home, tbey han take
(2500) log(f) = 5431.44 , i am very quiet. let the monthz and yearz come, they can take
(3000) log(f) = 5448.21 , i hm very quiet. let tae montas hnd yehrs come, taey chn thke
(3500) log(f) = 5448.21 , i am very wuiet. let the months and years come, they can take
------------------------------------------------------------
Proposed key: icznbkxgmprqtwfdyeoljvuahs 

i am very quiet. let the months and years come, they can take nothing from me, they can take nothing more. i am so alone, and so without hope that i can confront them without fear. the life that has borne me through these years is still in my hands and my eyes. whether i have subd

All Quiet On The Western Front, Chapter 12

# Knapsack Problem

In [433]:
def dotproduct(one,two):
    dot = []
    for i1, i2 in zip(one,two):
        dot.append(i1*i2)
    return sum(dot)

In [434]:
def toggle(cur_choice):
    s = random.randint(0,20)
    new_choice = cur_choice
    new_choice[s] = 1 - new_choice[s]
    if dotproduct(new_choice,weights) <= max_weight:
        return new_choice
    else:
        return toggle(cur_choice)

In [435]:
n_runs = 10
n_iters = 100000

values =[99, 26, 71, 8, 28, 54, 5, 40, 100, 91, 27, 16, 78, 20, 97, 18, 63,24, 1, 71]
weights =[19, 17, 86, 52, 34, 58, 35, 96, 14, 92, 56, 51, 92, 70, 20, 83, 20, 18, 14, 25]
max_weight = 200
beta = 0.0005
delta = 0.0005


for _ in range(n_runs): 
    # Initialize the current score to zero.
    # Generate a list with 0,1 and length 20.
    cur_log_f = 0 
    
    randomlist = []
    for i in range(20):
        n = random.randint(0,2)
        randomlist.append(n)
    cur_choice = randomlist

    for itr in range(n_iters):
        # Change 1 digit in the current choice.
        new_choice = toggle(cur_choice)

        # Score the new choice.
        new_log_f = beta * dotproduct(new_choice, values) 
        
        # If new score > current score, accept the new choice (current=new).
        # Else, with probability (new score)/(current score) accept the new choice.
        if new_log_f > cur_log_f: # redundant condition: just for clarity
            cur_choice = new_choice
            cur_log_f = new_log_f
        else:
            r = random.uniform(0,1)
            if r < np.exp(new_log_f-cur_log_f): 
                cur_choice = new_choice
                cur_log_f = new_log_f
        if itr%500==0 and itr != 0:
            beta += delta
        if itr%5000==0:
            print('(%d) log(f) = %.2f , '%(itr,cur_log_f) + 
                  'and combined value is ' + str(dotproduct(cur_choice,values)))
            print('The weight is as follows:')
            print(cur_choice)
      
    print('------------------------------------------------------------')
    print(dotproduct(cur_choice,values))  
    print('------------------------------------------------------------\n\n')

(0) log(f) = 0.17 , and combined value is 332
The weight is as follows:
[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1]
ERROR! Session/line number was not unique in database. History logging moved to new session 125
(5000) log(f) = 1.41 , and combined value is 281
The weight is as follows:
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]
(10000) log(f) = 3.45 , and combined value is 345
The weight is as follows:
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]
(15000) log(f) = 7.23 , and combined value is 228
The weight is as follows:
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
(20000) log(f) = 8.28 , and combined value is 137
The weight is as follows:
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1]
(25000) log(f) = 12.50 , and combined value is 256
The weight is as follows:
[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
(30000) log(f) = 10.89 , and combined value is 194
The weight is as follows:
[0, 1, 0,

(85000) log(f) = 148.45 , and combined value is 229
The weight is as follows:
[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
(90000) log(f) = 143.63 , and combined value is 290
The weight is as follows:
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0]
(95000) log(f) = 156.73 , and combined value is 239
The weight is as follows:
[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]
------------------------------------------------------------
86
------------------------------------------------------------


(0) log(f) = 94.78 , and combined value is 317
The weight is as follows:
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
(5000) log(f) = 162.07 , and combined value is 216
The weight is as follows:
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]
(10000) log(f) = 160.73 , and combined value is 305
The weight is as follows:
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]
(15000) log(f) = 160.73 , and combined value 

(70000) log(f) = 300.91 , and combined value is 332
The weight is as follows:
[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
(75000) log(f) = 300.91 , and combined value is 229
The weight is as follows:
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]
(80000) log(f) = 307.32 , and combined value is 280
The weight is as follows:
[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]
(85000) log(f) = 307.32 , and combined value is 246
The weight is as follows:
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(90000) log(f) = 307.32 , and combined value is 243
The weight is as follows:
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]
(95000) log(f) = 316.39 , and combined value is 292
The weight is as follows:
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
------------------------------------------------------------
163
------------------------------------------------------------


(0) log(f) = 66.32 , and combined valu

(55000) log(f) = 443.49 , and combined value is 178
The weight is as follows:
[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
(60000) log(f) = 445.72 , and combined value is 297
The weight is as follows:
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]
(65000) log(f) = 445.72 , and combined value is 206
The weight is as follows:
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
(70000) log(f) = 445.72 , and combined value is 154
The weight is as follows:
[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
(75000) log(f) = 462.71 , and combined value is 311
The weight is as follows:
[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
(80000) log(f) = 466.98 , and combined value is 140
The weight is as follows:
[0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
(85000) log(f) = 468.58 , and combined value is 91
The weight is as follows:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
(90000) log(f) = 468.58 , an

# Making 5-6 Letter Words that Sound Like Names

In [426]:
trigrams_df = pd.read_csv('names_trigrams.csv', index_col=0) * 1000
n_runs = 5
n_iters = 3000
lowercase = 'abcdefghijklmnopqrstuvwxyz'

In [427]:
def log_f(tgrm_df, msg):
    '''
    Parameters
    ----------
    tgrm_df : DataFrame with an index of trigrams 
              and a 'count' column.
    msg : A string of valud characters.
    Returns
    -------
    A log-score for the msg.
    '''    
    # Find pairs in the decrypted (using key) msg
    msg_df = pd.DataFrame() 
    for c1,c2,c3 in zip(msg[:-2], msg[1:-1], msg[2:]): 
        try:
            msg_df.loc[c1+c2+c3, 'count'] = msg_df.loc[c1+c2+c3, 'count']+1
        except: 
            msg_df.loc[c1+c2+c3, 'count'] = 1
                
    # For each trigram in the msg update log_f_score.    
    # If the trigram appears 0 times in the reference corpus,  
    #     subtract some small integer (log(k)<0 when k<1)          
    log_f_score = 0             
    for b in msg_df.index: 
        try: # succeed if tgrm_df.loc[b][0] > 0
            log_f_score += np.log(tgrm_df.loc[b][0])*msg_df.loc[b][0]
        except:
            log_f_score += -7 # tgrm_df.loc[b][0] = 0
            
    return log_f_score

In [428]:
for _ in range(n_runs): 
    # Initialize the current score to zero.
    # Generate a string of length 5 or 6.
    cur_log_f = 0 
    
    five = ''.join(rnd.sample(lowercase, 5))
    six = ''.join(rnd.sample(lowercase, 6))
    l = [five,six]
    cur_name = rnd.sample(l,1)[0]

    for itr in range(n_iters):
        # Change 1 random letter in the current name.
        c1 = rnd.sample(str(cur_name), 1)[0]
        d1 = rnd.sample(lowercase, 1)[0]
        new_name = cur_name.replace(c1,d1)
        
        # Score the new name.
        new_log_f = log_f(trigrams_df, new_name) 
        
        # If new score > current score, accept the new name (current=new).
        # Else, with probability (new score)/(current score) accept the new name.
        if new_log_f > cur_log_f: # redundant condition: just for clarity
            cur_name = new_name
            cur_log_f = new_log_f
        else:
            r = random.uniform(0,1)
            if r < np.exp(new_log_f-cur_log_f): 
                cur_name = new_name
                cur_log_f = new_log_f
        if itr%500==0:       
            print('(%d) log(f) = %.2f , '%(itr,cur_log_f) + new_name )
            
    # Print (or save) the current name.         
    print('------------------------------------------------------------')
    print(new_name)  
    print('------------------------------------------------------------\n\n')

(0) log(f) = 0.00 , jngtcv
(500) log(f) = 43.93 , nnnnnn
(1000) log(f) = 43.42 , snsnsn
(1500) log(f) = 43.93 , ononon
(2000) log(f) = 43.42 , ananan
(2500) log(f) = 43.93 , ononon
------------------------------------------------------------
acacac
------------------------------------------------------------


(0) log(f) = 4.96 , sarzwm
(500) log(f) = 45.03 , zrrzrz
(1000) log(f) = 46.82 , ejjeje
(1500) log(f) = 46.45 , rnnrnr
(2000) log(f) = 46.82 , xllxlx
(2500) log(f) = 45.03 , brrbrb
------------------------------------------------------------
essese
------------------------------------------------------------


(0) log(f) = 0.00 , yfovx
(500) log(f) = 37.28 , snasn
(1000) log(f) = 34.17 , gegge
(1500) log(f) = 35.01 , lhllh
(2000) log(f) = 34.29 , lgllg
(2500) log(f) = 35.01 , eeeee
------------------------------------------------------------
lilli
------------------------------------------------------------


(0) log(f) = 0.00 , xzqfyw
(500) log(f) = 43.90 , lllwnw
(1000) log(f) 